### Operación del <i>Pre-Regulador</i>

El <i>Pre-Regulador</i> esta ideado para trabajar en lazo abierto, con un ciclo de trabajo constante, capaz de suministrar la corriente máxima con la tensión mínima requerida por el (post)regulador lineal.<br>

Como la corriente de carga del pre-regulador es variable, al menos inicialmente tiene un valor muy bajo, la tensión a su salida es elevada, sobrepasando los límites de trabajo del (post)regulador lineal, por cual se debio incluir el diodo zener DZ1 no previsto inicialmente.   

Si se aplica el mismo ciclo de trabajo desde el arranque, la corriente en el MOSFET y la inductancia tienen un sobreimpulso inicial varias veces la corriente necesaria, este fenómeno se produce por el retardo del condensador de salida para alcanzar su tensión de trabajo, intervalo en el cual la tensión sobre la inductancia en la etapa de no conducción (del MOSFET) es menor a la de equilibrio, por lo cual no transfiere toda la energía ganada (en la etapa de conducción), almacenando progresiva- mente energía y aumentando su corriente. Este fenómeno se aminora calibrando la resistencia R1 de manera que la tensión inicial que genera sea aproximada a la tensión de trabajo, su valor es hallado experimentalmente, por cuanto la corriente de carga inicial es la suma del diodo zener (DZ1), la del regulador (IC1) y prinicpalmente del microcontrolador (IC2).<br>

### Ecuaciones de Operación del <i>Pre-Regulador</i>
Las ecuaciones ecuaciones que gobiernan el <i>pre-regulador</i> son :<br> <br> 
$$\begin{align}
  L\cdot\frac{di_L}{dt} &= V_{cc}      \\
  C\cdot\frac{dv_C}{dt} &= -I_{load} 
\end{align}\Bigg\}\quad\text{MOSFET conduciendo}
$$

$$\begin{align}
  L\cdot\frac{di_L}{dt} &= -V_D - v_C    \\
  C\cdot\frac{dv_C}{dt} &= i_L- I_{load}
\end{align}\Bigg\}\quad\text{MOSFET apagado}
$$

La solución cuando el MOSFET esta <i>conduciendo</i> son :<br>
$$\begin{align}
  i_{L(peak)}[k-1] &= i_L[k-1] + \frac{V_{cc}}{L}\cdot T_{on}     \\
  v_{C(dip)}[k-1] &= v_C[k-1] - \frac{I_{load} }{C}\cdot T_{on}
\end{align}
$$

La solución cuando el MOSFET esta <i>apagado</i> son :<br>
$$\begin{align}
  A[k-1] &= \frac{1}{2}(v_{C(dip)}[k-1] + V_D) - j \frac{L}{2}(i_{L(peak)}[k-1] - I_{load}) \\
  v_C[k] &= \mathcal{Re}\{A[k-1]\cdot e^{j\frac{T_{off}[k-1]}{LC}}\} - V_D \\
  i_L[k] &= \mathcal{Im}\{A[k-1]\cdot e^{j\frac{T_{off}[k-1]}{LC}}\} + I_{load} 
\end{align}
$$



### NCO
El <i>Pre-Regulador</i> es controlado por el NCO del microcontrolador operado en el modo <i>Pulse Frequency</i>. En este modo la la selección del tiempo de conducción es limitada ($T_{on}$), existiendo mucho mayor margen para la selección para el valor del periodo ($T_{on} + T_{off}$).

In [1]:
# Los módulos de soporte son los estándar, ...
%pylab inline
import matplotlib.pyplot as plt
from scipy.signal import *
import collections

# Cambia el tamaño de los gráficos, para ocupar todo el ancho de la página:
from pylab import rcParams
rcParams['figure.figsize'] = 16, 3


Populating the interactive namespace from numpy and matplotlib


#### Apendice : Solución del Sistema de ecuaciones con el MOSFET apagado
$$\begin{align}
  L\cdot\frac{di_L}{dt} &= -V_D - v_C     \tag{A.1} \\ 
  C\cdot\frac{dv_C}{dt} &= i_L- I_{load}  \tag{A.2} 
\end{align}
$$

Derivando la segunda ecuación (A.2) y eliminando $\frac{d}{dt}i_L$ se obtiene : 
$$LC\cdot\frac{d}{dt}^2v_C + v_C = -V_D  \tag{A.3}$$

cuya solución general es : <br>
$$v_C = A\cdot e^{j\cdot \frac{t}{LC}} + B\cdot e^{-j\cdot \frac{t}{LC}} - V_D     \tag{A.4}$$

Como $v_C$ es real, no es dificíl deducir que $A$ y $B$ son conjugados : <br>
$$v_C = A\cdot e^{j\cdot \frac{t}{LC}} + A^*\cdot e^{-j\cdot \frac{t}{LC}} - V_D     \tag{A.5a}$$
$$v_C = 2\cdot\mathcal{Re}\{A\cdot e^{j\cdot \frac{t}{LC}}\} - V_D                       \tag{A.5b}$$


reemplazando la A.4 en A.2 : <br>
$$i_L = j\cdot \frac{A}{L}\cdot e^{j\cdot \frac{t}{LC}} -j\cdot \frac{A^*}{L}\cdot e^{-j\cdot \frac{t}{LC}} + I_{load}  \tag{A.6a}$$
$$i_L = \frac{2}{L}\cdot\mathcal{Im}\{A\cdot e^{j\cdot \frac{t}{LC}}\} + I_{load}                       \tag{A.6b}$$


La constante A puede hallarse de las condiciones iniciales para t = 0 en las ecuaciones A.5b y A.6b :<br>
$$\begin{align}
  v_C(0) &= 2\cdot\mathcal{Re}\{A\} - v_D  \\
  i_L(0) &= \frac{2}{L}\cdot\mathcal{Im}\{A\} + I_{load}
\end{align}
$$

de donde se deduce :<br>
$$A = \frac{1}{2}(v_C(0) + v_D) + j\frac{L}{2}(i_L(0) - I_{load})$$

$$A = \sqrt{(v_C(0) + v_D)^2 + (i_L(0) - I_{load})^2}\cdot e^{j\cdot \arctan\Big(L\cdot\frac{i_L(0) - I_{load}}{v_C(0) + v_D}\Big)}$$

finalmente :<br>
$$\begin{align}
  v_C(t) = \sqrt{(v_C(0) + v_D)^2 + (i_L(0) - I_{load})^2}\cdot \cos\Big( \frac{t}{LC}+ \arctan\Big(L\cdot\frac{i_L(0) - I_{load}}{v_C(0) + v_D}\Big)\Big) - V_D \\
  i_L(t) = \sqrt{(v_C(0) + v_D)^2 + (i_L(0) - I_{load})^2}\cdot \sin\Big( \frac{t}{LC}+ \arctan\Big(L\cdot\frac{i_L(0) - I_{load}}{v_C(0) + v_D}\Big)\Big) + I_{load}
\end{align}$$
